In [2]:
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType

feats = []
f = open('features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

print(schema)

StructType(List(StructField(Timestamp,LongType,true),StructField(Geohash,StringType,true),StructField(geopotential_height_lltw,FloatType,true),StructField(water_equiv_of_accum_snow_depth_surface,FloatType,true),StructField(drag_coefficient_surface,FloatType,true),StructField(sensible_heat_net_flux_surface,FloatType,true),StructField(categorical_ice_pellets_yes1_no0_surface,FloatType,true),StructField(visibility_surface,FloatType,true),StructField(number_of_soil_layers_in_root_zone_surface,FloatType,true),StructField(categorical_freezing_rain_yes1_no0_surface,FloatType,true),StructField(pressure_reduced_to_msl_msl,FloatType,true),StructField(upward_short_wave_rad_flux_surface,FloatType,true),StructField(relative_humidity_zerodegc_isotherm,FloatType,true),StructField(categorical_snow_yes1_no0_surface,FloatType,true),StructField(u-component_of_wind_tropopause,FloatType,true),StructField(surface_wind_gust_surface,FloatType,true),StructField(total_cloud_cover_entire_atmosphere,FloatType,tru

In [3]:

#df = spark.read.format('csv').option('sep', '\t').schema(schema).load('/Volumes/evo/Datasets/NAM_2015_S/*')
df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/nam_tiny.tdv')
# df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/nam_201501.tdv.gz')


In [4]:
df_large = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/nam_201509.tdv.gz')


In [32]:
df_entire = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/*')


In [5]:
really_hot = df.filter(df.snow_depth_surface != 0).count()
print(really_hot)

hot_and_humid = df.filter(df.temperature_surface > 313).filter(df.relative_humidity_zerodegc_isotherm > .8).count()
print(hot_and_humid)

25
0


In [8]:
# df.filter(df.snow_depth_surface != 0).take(5)

[Row(Timestamp=1426377600000, Geohash='cf7ecr4h2ps0', geopotential_height_lltw=136.53125, water_equiv_of_accum_snow_depth_surface=77.0, drag_coefficient_surface=0.0, sensible_heat_net_flux_surface=-39.57763671875, categorical_ice_pellets_yes1_no0_surface=0.0, visibility_surface=24221.587890625, number_of_soil_layers_in_root_zone_surface=3.0, categorical_freezing_rain_yes1_no0_surface=0.0, pressure_reduced_to_msl_msl=99602.0, upward_short_wave_rad_flux_surface=6.625, relative_humidity_zerodegc_isotherm=34.0, categorical_snow_yes1_no0_surface=0.0, u-component_of_wind_tropopause=27.527877807617188, surface_wind_gust_surface=16.158788681030273, total_cloud_cover_entire_atmosphere=100.0, upward_long_wave_rad_flux_surface=314.0560302734375, land_cover_land1_sea0_surface=1.0, vegitation_type_as_in_sib_surface=18.0, v-component_of_wind_pblri=12.31915283203125, albedo_surface=38.75, lightning_surface=0.0, ice_cover_ice1_no_ice0_surface=0.0, convective_inhibition_surface=-0.65234375, pressure_su

# Warn_up

## Q1. Unknown Feature: Choose a feature from the data dictionary above that you have never heard of before. Inspect some of the values for the feature (such as its average, min, max, etc.) and try to guess what it measures. Was your hypothesis correct?

In [9]:
from pyspark.sql.functions import avg

df.select(avg(df.geopotential_height_pblri)).show()

+------------------------------+
|avg(geopotential_height_pblri)|
+------------------------------+
|              406.451460521314|
+------------------------------+



This attribute may records geopotential height for each loaction.

## Q2. Hot hot hot: When and where was the hottest temperature observed in the dataset? Is it an anomaly?

In [10]:
# Creating an SQL 'table'
df.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
hot = spark.sql("SELECT Geohash,temperature_surface FROM TEMP_DF WHERE temperature_surface IN (SELECT MAX(temperature_surface) FROM TEMP_DF) ").collect()

for x in hot:
    print(x)


Row(Geohash='9et0wfypj3zb', temperature_surface=321.3058776855469)


The hottest temperature in this dataset is 321.3F (160.7C). Obviously, Human cannot live in such hot place.

## Q3. So Snowy: Find a location that is snowy all year (there are several). Locate a nearby town/city and provide a small writeup about it. Include pictures if you’d like.

In [ ]:
# Creating an SQL 'table'
df_large.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
snow = spark.sql("SELECT Geohash FROM TEMP_DF WHERE Geohash NOT IN (SELECT DISTINCT Geohash FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0) ").collect()
for x in snow:
    print(x)

In [39]:
# Creating an SQL 'table'
df_large.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
# not_snow_all_year = spark.sql("SELECT DISTINCT Geohash FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0 ").collect()
# all_Geohash = spark.sql("SELECT DISTINCT Geohash FROM TEMP_DF").collect()
snow = spark.sql("SELECT DISTINCT Geohash from TEMP_DF group by Geohash having avg(snow_cover_surface) >= 100").collect()
# snow = spark.sql("SELECT count(DISTINCT Geohash) from TEMP_DF group by Geohash having avg(categorical_snow_yes1_no0_surface) = 1").collect()
for x in snow:
    print(x)

# not_snow_all_year = spark.sql("SELECT Geohash, count(*) FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0 group by Geohash").collect()
# all_Geohash = spark.sql("SELECT Geohash, count(*) FROM TEMP_DF group by Geohash").collect()
# not_snow_all_year.show()
# snow = df.filter(df.categorical_snow_yes1_no0_surface == 0).take(5)
# categorical_snow_yes1_no0_surface
# print(snow)
# snow_count = df.filter(df.Geohash == '9et0wfypj3zb').count()
# print ('Count in one month: ', snow_count)
# .collect() gives us a list of rows. Let's grab the first 10:
# for x in not_snow_all_year:
#     print(x)

# for x in all_Geohash:
#     print(x)
# print(len(all_Geohash))

# for i in range(len(all_Geohash))
#     if 
    
# What's the maximum value?
# snowmax = spark.sql("SELECT MAX(snow_depth_surface) as maxval FROM TEMP_DF").collect()

# print('Max val observed:', snowmax)

Row(Geohash='dpvgjz6dbkzz')
Row(Geohash='f1k4351udmzz')
Row(Geohash='drcmh4102rxb')
Row(Geohash='f2fkht7j67kp')
Row(Geohash='f0872jfzps80')
Row(Geohash='c3ft7x0fccup')
Row(Geohash='f1rkwfrs3xeb')
Row(Geohash='cdpsb0v00qrz')
Row(Geohash='f36nhnz43tup')
Row(Geohash='f6w19bx1pzxb')
Row(Geohash='c6kyf5k1z9eb')
Row(Geohash='c6q3z21rcnh0')
Row(Geohash='cddfhshhn45b')
Row(Geohash='c9bp0mwdw9bp')
Row(Geohash='f6ksr5hf4sbp')
Row(Geohash='f41s0prbqepb')
Row(Geohash='f6pmrrpq24bp')
Row(Geohash='c3ntve3jyfgz')
Row(Geohash='f9zsb9cwqr80')
Row(Geohash='c3qx0k7pk7rz')
Row(Geohash='f3v1tcjhp8kp')
Row(Geohash='cdjb1hxp5z7z')
Row(Geohash='f2xsrw2frvgz')
Row(Geohash='fd6syrqm63yp')
Row(Geohash='cbxb1em977xb')
Row(Geohash='f1n5uuj79j5b')
Row(Geohash='f3ujscs2v0xb')
Row(Geohash='f2s9jg2t8rzz')
Row(Geohash='cd0vmnh30q5b')
Row(Geohash='f13t6fjqgfh0')
Row(Geohash='fdhwjq1bq3w0')
Row(Geohash='f1e8d0dp3xs0')
Row(Geohash='f35s2f54q2pb')
Row(Geohash='ccw0e5vwqnh0')
Row(Geohash='cd4yce2vq7s0')
Row(Geohash='f6nn1sr

Row(Geohash='f1xx7epyk3xb')
Row(Geohash='f1rj359057rz')
Row(Geohash='cddexd2b8qh0')
Row(Geohash='f37t4tph00s0')
Row(Geohash='cdz5fdgr1rrz')
Row(Geohash='c8z82bgevus0')
Row(Geohash='f0hq4bh7jr2p')
Row(Geohash='cfhwu5uew5kp')
Row(Geohash='c6wqb5nbk1eb')
Row(Geohash='f324qxztjkxb')
Row(Geohash='c6tkf8drug5b')
Row(Geohash='9rzq8ugsekrz')
Row(Geohash='f27urzzc32xb')
Row(Geohash='f36xrtz6nb80')
Row(Geohash='f27hssxeexbp')
Row(Geohash='9wsns5b1c4zz')
Row(Geohash='f6j5nebhdr7z')
Row(Geohash='cchvy2k87heb')
Row(Geohash='cfwfzm4rzceb')
Row(Geohash='dpgxby5b90s0')
Row(Geohash='ccx5hqtzp1eb')
Row(Geohash='f6d4p1bfdbkp')
Row(Geohash='f4dq79suw02p')
Row(Geohash='cbxmv5eqm380')
Row(Geohash='f0yqxtqptbeb')
Row(Geohash='cd74d6edty5b')
Row(Geohash='fd4gfhv7cmgz')
Row(Geohash='c6hnthjwdcbp')
Row(Geohash='cd8sdg7uxg7z')
Row(Geohash='9rggd6y0xbrz')
Row(Geohash='f3tzm3z17h7z')
Row(Geohash='cf8s1x7cwtpb')
Row(Geohash='ccwgv0ydzgxb')
Row(Geohash='cbw4wge4ms2p')
Row(Geohash='9x8rg8swy3zz')
Row(Geohash='f3dmx5k

Row(Geohash='f1mgxnrqzzup')
Row(Geohash='c2n0519y0400')
Row(Geohash='f27kqu0kvbxb')
Row(Geohash='cdxp2t59hukp')
Row(Geohash='c3xdjgrcqtzz')
Row(Geohash='cc5eec4snj5b')
Row(Geohash='cc55d0h79pup')
Row(Geohash='f0xg9nbgf1kp')
Row(Geohash='c1r3gv5spm00')
Row(Geohash='c6wrfuxfmq00')
Row(Geohash='cfrtm91rew2p')
Row(Geohash='f0u8z8qphhs0')
Row(Geohash='c30cgkd5rskp')
Row(Geohash='c1wb533zdg7z')
Row(Geohash='c6s3ddtwtkxb')
Row(Geohash='f2gh1z3bg1up')
Row(Geohash='c1rd4usn07xb')
Row(Geohash='c8xx1hybq2h0')
Row(Geohash='cf2pekkv5rrz')
Row(Geohash='f6w6bhb79sbp')
Row(Geohash='c3ct8grwq7pb')
Row(Geohash='9x924g2pgzkp')
Row(Geohash='f92b2yehjqxb')
Row(Geohash='cdxbx5u57zs0')
Row(Geohash='cbf0prw0qz7z')
Row(Geohash='dryjd2cp7880')
Row(Geohash='f6q8ck3j0n5b')
Row(Geohash='cf6vcybcz1s0')
Row(Geohash='f2jzbue9zvzz')
Row(Geohash='c95x43y9c9kp')
Row(Geohash='c8wjsfuj1skp')
Row(Geohash='cfvhuswk392p')
Row(Geohash='c43bfb4h3b00')
Row(Geohash='c44vmb5bjkzz')
Row(Geohash='f1scdvhswe2p')
Row(Geohash='f0zj227

Row(Geohash='cdwru2jgpkrz')
Row(Geohash='c9jp8tjggrxb')
Row(Geohash='c4xb114439eb')
Row(Geohash='drgwn48wkceb')
Row(Geohash='cdyg6c8et5eb')
Row(Geohash='f110pmsd4g2p')
Row(Geohash='cdu0h46jhx80')
Row(Geohash='f2m4ny97qreb')
Row(Geohash='c32z7e0rx1up')
Row(Geohash='cd2u4gybsjgz')
Row(Geohash='9rynzr3qsvup')
Row(Geohash='cdy65ntd6rkp')
Row(Geohash='f4w1edqt8kzz')
Row(Geohash='c1wq471dw5eb')
Row(Geohash='cb9vvmvcbjbp')
Row(Geohash='f2er7mc6n27z')
Row(Geohash='cfx57ku1pr2p')
Row(Geohash='cc61kzkssmzz')
Row(Geohash='c6j1d83hw9bp')
Row(Geohash='f47mqqyqtekp')
Row(Geohash='cd5jqnfnzt5b')
Row(Geohash='cbbry1e8qheb')
Row(Geohash='c2ww98x5rtgz')
Row(Geohash='f6h0c479cypb')
Row(Geohash='c3jmw242t4pb')
Row(Geohash='c9jnsg34fdpb')
Row(Geohash='cdtz0g0u5e2p')
Row(Geohash='f9hjwjhvpuw0')
Row(Geohash='c2pw2nyjnzkp')
Row(Geohash='cdezhb1y1rzz')
Row(Geohash='f1nwvq20yqxb')
Row(Geohash='cczj6dty97xb')
Row(Geohash='9xb01s32qmgz')
Row(Geohash='f23815pr77rz')
Row(Geohash='f6xms65z7rxb')
Row(Geohash='f0f6z88

Row(Geohash='f3vtqjmpjrxb')
Row(Geohash='f2k6upx0j4h0')
Row(Geohash='f96nnnesmz7z')
Row(Geohash='c3gexzp535xb')
Row(Geohash='f07vdpzndqh0')
Row(Geohash='fd0nvr2ts7rz')
Row(Geohash='f9pqzwy7ukkp')
Row(Geohash='cbmhuqunrurz')
Row(Geohash='c6tmc24sm92p')
Row(Geohash='f98qx73t3pxb')
Row(Geohash='fb8rhcuhht1b')
Row(Geohash='f8808ude3280')
Row(Geohash='cc0gg0xy2h80')
Row(Geohash='f0c5wn836ps0')
Row(Geohash='fd4r95cxt880')
Row(Geohash='c98qru4edwxb')
Row(Geohash='9xf5rubcc05b')
Row(Geohash='drfe6xfhcjpb')
Row(Geohash='f6hpt3g3gzkp')
Row(Geohash='f2cnxvysg1up')
Row(Geohash='c2gxkprehgh0')
Row(Geohash='f3pkbmzu32pb')
Row(Geohash='f1fzpk3dh4h0')
Row(Geohash='f3jxh8qtk1gz')
Row(Geohash='cbqe0hm5q0s0')
Row(Geohash='c6yc1sy2zgkp')
Row(Geohash='c367u6hf6kxb')
Row(Geohash='cd1s1qk925s0')
Row(Geohash='fd0skvdppurz')
Row(Geohash='c9g1ys8r2h2p')
Row(Geohash='f2q5eszw7980')
Row(Geohash='f0ydp1j38cbp')
Row(Geohash='cf69ykq33b7z')
Row(Geohash='cdwp0tr8szs0')
Row(Geohash='f6s5cq8jdn80')
Row(Geohash='cbfcrw1

...

Row(Geohash='f63wgejr3pkp')
Row(Geohash='c900jm1g3ukp')
Row(Geohash='f489wjmfzj5b')
Row(Geohash='c6e7gqj4tx2p')
Row(Geohash='c9hrqyh7q2kp')
Row(Geohash='f4funcb9ys2p')
Row(Geohash='9xvsx4hf64gz')
Row(Geohash='cd2egrb6mdup')
Row(Geohash='cchrkt77f7pb')
Row(Geohash='f68bm2jc86h0')
Row(Geohash='f104su986sxb')
Row(Geohash='c4nv5xxdz3bp')
Row(Geohash='cbr6qmsf7e00')
Row(Geohash='fd5e0mdg0dgz')
Row(Geohash='f2s19t3vykeb')
Row(Geohash='cfkr891h5ugz')
Row(Geohash='cdhs2vetvhkp')
Row(Geohash='f96xq9wmqu6p')
Row(Geohash='cc20se4cdbs0')
Row(Geohash='ccccdu678v00')
Row(Geohash='f32wt5cgxb7z')
Row(Geohash='f0y01h3sy4h0')
Row(Geohash='f33ym59w98eb')
Row(Geohash='c6k9jddjuyh0')
Row(Geohash='c9zjzmm6fuup')
Row(Geohash='c6ks95kmw1bp')
Row(Geohash='c6w63dbwdp80')
Row(Geohash='cfb28hmhwheb')
Row(Geohash='drgvzcb8792p')
Row(Geohash='c9fr23d9bfup')
Row(Geohash='f0vrv0gfbqeb')
Row(Geohash='c2n3bns6t5xb')
Row(Geohash='c1xj4rcxdb80')
Row(Geohash='f1ggetf0h880')
Row(Geohash='cf39g8z3t8kp')
Row(Geohash='ccv9v2k

Row(Geohash='f2ynb8y79yeb')
Row(Geohash='cfn4jkd371up')
Row(Geohash='c91w49mhhzgz')
Row(Geohash='f4xmwb4m87eb')
Row(Geohash='c6tegwhn7u7z')
Row(Geohash='f3m11qy8qx2p')
Row(Geohash='c9btbu10cnxb')
Row(Geohash='f6r8rubsr9eb')
Row(Geohash='9qerb82uue80')
Row(Geohash='f49mp0qkfpup')
Row(Geohash='ccrf7vnvgjup')
Row(Geohash='f1u2k36pmvup')
Row(Geohash='c9uu9hyjmueb')
Row(Geohash='dpffgjecbsxb')
Row(Geohash='f6pu1hvswgkp')
Row(Geohash='f95ysej5d7tb')
Row(Geohash='f89qb3c7u92p')
Row(Geohash='c8yc5xp8fubp')
Row(Geohash='ccpd5jbv2dpb')
Row(Geohash='f1wzjm9ydgkp')
Row(Geohash='c2x0ssemeykp')
Row(Geohash='f24g96gdyvh0')
Row(Geohash='f9cdzc0u8400')
Row(Geohash='f8rxvkyukmup')
Row(Geohash='cdh4dwmtq3zz')
Row(Geohash='f4myt7z5h1s0')
Row(Geohash='f0pq9re2cfh0')
Row(Geohash='f3m9xyukxhs0')
Row(Geohash='f0ws2p50bykp')
Row(Geohash='f82ujg4u5hup')
Row(Geohash='c6p9532ve3zz')
Row(Geohash='c2brsnpg73zz')
Row(Geohash='f4h66eq190rz')
Row(Geohash='fdjy980gyzrz')
Row(Geohash='c2f9kj4wk05b')
Row(Geohash='f24qgk1

Row(Geohash='c9j9jxb1f12p')
Row(Geohash='f28epdu6zykp')
Row(Geohash='c9sep07cnkkp')
Row(Geohash='f01ew2s3rf5b')
Row(Geohash='f190ttc2fqxb')
Row(Geohash='9rvxdex6epeb')
Row(Geohash='f2tebchr4ukp')
Row(Geohash='cdy7wqqh6000')
Row(Geohash='f3bmr303jr2p')
Row(Geohash='cd0eentmnrs0')
Row(Geohash='f85p5100gn9b')
Row(Geohash='cdwm38p43xeb')
Row(Geohash='f0xh9x3k1f5b')
Row(Geohash='cf1zgb57m780')
Row(Geohash='drdsgv0s2bh0')
Row(Geohash='f8zqe0pznh3z')
Row(Geohash='c8ge64rd6k7z')
Row(Geohash='cd4ur4z11npb')
Row(Geohash='f3hdg5zpkfh0')
Row(Geohash='cbb38fbs0bkp')
Row(Geohash='drswb2p33cup')
Row(Geohash='f3dfn0c2g0eb')
Row(Geohash='c3d4rcfmnreb')
Row(Geohash='c81461xjypeb')
Row(Geohash='cfb5p6c0m4up')
Row(Geohash='f410xkd27heb')
Row(Geohash='cf008ycp6u2p')
Row(Geohash='cceu6nqqxkxb')
Row(Geohash='cdeybyf9jhkp')
Row(Geohash='dprt8x8dbmpb')
Row(Geohash='f3ntrncc5dbp')
Row(Geohash='cftd5bd5uc7z')
Row(Geohash='cc8udtv6n52p')
Row(Geohash='f3185gcc1vup')
Row(Geohash='c9h9mt7rqp2p')
Row(Geohash='c90mrke

Row(Geohash='cc2qdh9djgkp')
Row(Geohash='fd01pftdhd00')
Row(Geohash='c6eu710jr5xb')
Row(Geohash='c9x3hhkxgpkp')
Row(Geohash='cfh7b8w6bwxb')
Row(Geohash='cbs730xc0000')
Row(Geohash='f16dy41we0eb')
Row(Geohash='f46bs2dndreb')
Row(Geohash='f4txb9u85qrz')
Row(Geohash='f2cue647qu2p')
Row(Geohash='c93uk910gsbp')
Row(Geohash='f323cu7skbh0')
Row(Geohash='c6e6uy9sybeb')
Row(Geohash='f84h5smeu5h0')
Row(Geohash='fd2ju17tk4h0')
Row(Geohash='c1vy0zrdukzz')
Row(Geohash='c1rudk5jzypb')
Row(Geohash='f20bmg5e387z')
Row(Geohash='f1rxc6p0kzkp')
Row(Geohash='f95jcuvm1x6p')
Row(Geohash='f0yeub9gzckp')
Row(Geohash='c3crwj3erxkp')
Row(Geohash='c4kqrs5x6780')
Row(Geohash='f9r7tk36q51b')
Row(Geohash='c44qhn77kfpb')
Row(Geohash='dpwxjz3xuheb')
Row(Geohash='f4dj75zegtbp')
Row(Geohash='f20ce6tbcy7z')
Row(Geohash='f2b2d2jgk57z')
Row(Geohash='f99m5mf663rz')
Row(Geohash='f613190wcnrz')
Row(Geohash='f4pk5qhgzbh0')
Row(Geohash='cc9j3bhnrx7z')
Row(Geohash='cfybkkpqp57z')
Row(Geohash='cdkkxsu6hqrz')
Row(Geohash='f02951h

Row(Geohash='f9tvec735c3z')
Row(Geohash='cbm0ztf74fgz')
Row(Geohash='f48ddw4cxe80')
Row(Geohash='f062rrxf7yh0')
Row(Geohash='c3u33z16pg7z')
Row(Geohash='c6wf9h5vprxb')
Row(Geohash='cf17t3zp7rxb')
Row(Geohash='cbmj912tgheb')
Row(Geohash='c1v1vb2pjg5b')
Row(Geohash='f3y761mcde80')
Row(Geohash='9zyq7k4jrbxb')
Row(Geohash='cf73mtk6qkkp')
Row(Geohash='c38bcq1gvseb')
Row(Geohash='f661mkd2m9kp')
Row(Geohash='cb8xgx48derz')
Row(Geohash='cfcb276sd3gz')
Row(Geohash='9rzddrycke80')
Row(Geohash='f2nsb8wfp7xb')
Row(Geohash='cfbgxq45r22p')
Row(Geohash='cbfnfyxhegs0')
Row(Geohash='f4m5zfsdypup')
Row(Geohash='cdb3tbq3kqpb')
Row(Geohash='f9k8yxtgrzw0')
Row(Geohash='c65wv3jwrseb')
Row(Geohash='f48xskcfds80')
Row(Geohash='c9nf5ypf5pup')
Row(Geohash='f649pqpje2rz')
Row(Geohash='c33sv1e4xxkp')
Row(Geohash='f1nbb0tnu07z')
Row(Geohash='f9gnp7n1rw1b')
Row(Geohash='c2ymr1jhyt00')
Row(Geohash='cf8j1kq58hgz')
Row(Geohash='cbsx5xd42dbp')
Row(Geohash='f9hekezm66jb')
Row(Geohash='f8z4xfs37rmz')
Row(Geohash='c43f77g

# Analysis

## Q1. Strangely Snowy: Find a location that contains snow while its surroundings do not. Why does this occur? Is it a high mountain peak in a desert?

In [7]:
df_large.createOrReplaceTempView("TEMP_DF")
not_snow_all_year = spark.sql("SELECT  Geohash,avg(snow_cover_surface) FROM TEMP_DF group by Geohash having avg(snow_cover_surface) >= 99").collect()

for x in not_snow_all_year:
    print(x)
print(len(not_snow_all_year))

Row(Geohash='c1gxsefbz52p', avg(snow_cover_surface)=100.0)
Row(Geohash='c43hd76g0npb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43g3722yrzz', avg(snow_cover_surface)=100.0)
Row(Geohash='c1nt970j5b5b', avg(snow_cover_surface)=100.0)
Row(Geohash='c1grzzxwt1bp', avg(snow_cover_surface)=100.0)
Row(Geohash='c436f3k4p8xb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43jhzrrsbrz', avg(snow_cover_surface)=100.0)
Row(Geohash='c44hk1hy9u2p', avg(snow_cover_surface)=100.0)
Row(Geohash='c46sf3s53p80', avg(snow_cover_surface)=100.0)
Row(Geohash='c43gkzxxn6bp', avg(snow_cover_surface)=100.0)
Row(Geohash='c41tugrtwhzz', avg(snow_cover_surface)=100.0)
Row(Geohash='c458r20zx080', avg(snow_cover_surface)=100.0)
Row(Geohash='c43gb4ywdgrz', avg(snow_cover_surface)=100.0)
Row(Geohash='c438x5esgf00', avg(snow_cover_surface)=100.0)
Row(Geohash='c4459s8rz1xb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43c1zbmntbp', avg(snow_cover_surface)=100.0)
Row(Geohash='c43e1f64v9bp', avg(snow_cover_surface)=100.

In [8]:
four_set = set()
for x in not_snow_all_year:
    print(x.Geohash[0:4])
    four_set.add(x.Geohash[0:4])
print(len(not_snow_all_year))
print(len(four_set))

c1gx
c43h
c43g
c1nt
c1gr
c436
c43j
c44h
c46s
c43g
c41t
c458
c43g
c438
c445
c43c
c43e
c41z
c43u
c44n
c41u
c439
c1q2
c1nt
c432
c43e
c1gr
c44n
c1gn
c43e
c41z
c452
c1nm
c1nr
c44p
c43c
c304
c43b
c460
c43m
c43h
c1gy
c43k
c1gr
c43d
c1gz
c1gx
c43u
c44p
c43d
c41t
c433
c44j
c1q0
c41x
c436
c438
c1pc
c1gx
c43f
c437
c439
c43d
c459
c1nq
c43c
c1nk
c1p5
c460
c436
c43s
c1gz
c1p8
c464
c41v
c3k8
c43k
c45b
c41u
c3hz
c41u
c304
c1pe
c41w
c445
c438
c3k0
c3k3
c43b
c41w
c436
c43h
c460
c43u
c458
c307
c458
c1pf
c437
c464
c44h
c36u
c41g
c44p
c1uh
c44h
c1gy
c43g
c3k2
c461
c41x
c41z
c304
c1nu
c41y
c1gy
c41z
c41w
c301
c36u
c1sn
c43j
c44h
c44w
c43k
c464
c1gq
c1gw
c301
c465
c44n
c43e
c43f
c439
c1nk
c1gz
c439
c1gr
c41v
c43s
c43h
c1sn
c41y
c1gq
c41v
c43f
c4r5
c1p4
c1gz
149
74


In [24]:
# df.createOrReplaceTempView("TEMP_DF")
groupList = list()
for y in four_set:
        param = y + '%'
        group = spark.sql("SELECT distinct Geohash,avg(snow_cover_surface) as avgSnow FROM TEMP_DF WHERE Geohash like '%s' group by Geohash" %param ).collect()
        print(param ) 
        print(len(group))
        groupList.append(group)
        
print(len(groupList))
print('Complete')


c3hz%
4
c43g%
5
c43m%
3
c1p4%
5
c41z%
4
c36u%
4
c1nr%
4
c460%
3
c1p8%
3
c41g%
4
c436%
4
c1nt%
4
c433%
4
c41x%
4
c452%
3
c1q0%
4
c3k8%
4
c43e%
4
c43f%
3
c1uh%
4
c458%
4
c1nm%
3
c41y%
3
c43b%
5
c459%
4
c43u%
4
c1pf%
3
c3k3%
4
c44n%
4
c1gz%
4
c1p5%
3
c44h%
5
c1pc%
4
c1nu%
4
c432%
5
c44j%
4
c1nk%
5
c439%
4
c304%
5
c1gn%
4
c43s%
4
c3k0%
5
c43h%
4
c3k2%
3
c44w%
3
c43k%
4
c41w%
5
c461%
4
c43c%
4
c1gx%
3
c43d%
4
c1gr%
5
c44p%
4
c1nq%
4
c41v%
5
c1gq%
4
c1gw%
4
c41u%
4
c438%
3
c1gy%
4
c1sn%
4
c43j%
2
c41t%
4
c445%
4
c1q2%
5
c4r5%
4
c46s%
4
c45b%
4
c301%
4
c464%
5
c465%
4
c1pe%
4
c307%
4
c437%
4
74
Complete


In [26]:
for x in groupList:
    avg=0
    for y in x:
        avg= avg + y.avgSnow
    avg= avg/len(x)
    if avg<30:
        print(x)
#     print(avg)
        

[Row(Geohash='c1uhvnv5p8pb', avgSnow=7.283950617283951), Row(Geohash='c1uh66r1r02p', avgSnow=9.74074074074074), Row(Geohash='c1uhcd9k1n2p', avgSnow=100.0), Row(Geohash='c1uhqne01400', avgSnow=2.9135802469135803)]
[Row(Geohash='c3k0fgxqugeb', avgSnow=10.790123456790123), Row(Geohash='c3k0yw8m6us0', avgSnow=100.0), Row(Geohash='c3k0rj7e0zs0', avgSnow=21.506172839506174), Row(Geohash='c3k07d6swzkp', avgSnow=4.08641975308642), Row(Geohash='c3k00p3vw3gz', avgSnow=4.08641975308642)]


c1uhcd9k1n2p is a mountain peak(lavender peak)

## Q2. Lightning rod: Where are you most likely to be struck by lightning? Use a precision of at least 4 Geohash characters and provide the top 3 locations.

In [36]:
df_entire.createOrReplaceTempView("TEMP_DF")
lightning = spark.sql("SELECT  Geohash,avg(lightning_surface) as avgLighting FROM TEMP_DF group by Geohash order by avgLighting DESC Limit 10 ").collect()

for x in lightning:
    print(x)
print(len(lightning))

Row(Geohash='9eqepuxk7x20', avgLighting=0.3724035608308605)
Row(Geohash='9g3h968ygj7z', avgLighting=0.36053412462908013)
Row(Geohash='9g3y52sgeceb', avgLighting=0.3508902077151335)
Row(Geohash='9g3ug8ckk4hp', avgLighting=0.3486646884272997)
Row(Geohash='9g3m79nrf3zb', avgLighting=0.3479228486646884)
Row(Geohash='9g3mq3f7y6eb', avgLighting=0.3479228486646884)
Row(Geohash='9g3v7kxpuhh0', avgLighting=0.3464391691394659)
Row(Geohash='d7mkkfvu34up', avgLighting=0.336053412462908)
Row(Geohash='9g2vntcg8c2p', avgLighting=0.3353115727002967)
Row(Geohash='9g35cqggdn5z', avgLighting=0.33456973293768544)
10


I calculate the average lightning chances. The top 3 location is 9eqe, 9g3h, 9g3y

## Q5. Escaping the fog: After becoming rich from your startup, you are looking for the perfect location to build your Bay Area mansion with unobstructed views. Find the locations that are the least foggy and show them on a map.

In [40]:
df_entire.createOrReplaceTempView("TEMP_DF")
visibility = spark.sql("SELECT  Geohash,avg(visibility_surface) as avgVisibility FROM TEMP_DF WHERE Geohash like '9q9p%' or Geohash like '9q9n%' or Geohash like '9q9j%' or Geohash like '9q9m%' or Geohash like '9q9k%' or Geohash like '9q9h%' or Geohash like '9q8v%' or Geohash like '9q8y%' or Geohash like '9q8z%' group by Geohash order by avgVisibility DESC ").collect()
for x in visibility:
    print(x)
print(len(visibility))

Row(Geohash='9q9jnpfx5rxb', avgVisibility=23811.522961255705)
Row(Geohash='9q9jv7p453zz', avgVisibility=23703.065985376827)
Row(Geohash='9q9m35ffeg7z', avgVisibility=23698.021476396112)
Row(Geohash='9q9kc8wprss0', avgVisibility=23675.469547196735)
Row(Geohash='9q9jdrny12rz', avgVisibility=23616.419106960297)
Row(Geohash='9q8zuzx3hzeb', avgVisibility=23600.692098283627)
Row(Geohash='9q9k4m9efqh0', avgVisibility=23497.428004600173)
Row(Geohash='9q9hp3c2xy5b', avgVisibility=23449.653520560052)
Row(Geohash='9q9hwsyh477z', avgVisibility=23436.152038374712)
Row(Geohash='9q9mbqzmqxrz', avgVisibility=23435.8553033342)
Row(Geohash='9q9pesk2gbs0', avgVisibility=23400.098629552464)
Row(Geohash='9q9j5h47k07z', avgVisibility=23329.623851402575)
Row(Geohash='9q9py8hurvup', avgVisibility=23277.991815833026)
Row(Geohash='9q9nky5sd52p', avgVisibility=23164.93542607814)
Row(Geohash='9q9nxfh005kp', avgVisibility=23088.525926889935)
Row(Geohash='9q9p887u91up', avgVisibility=22978.882014243467)
Row(Geohash

The list above has the best visibility in the bay area. 
Such as 9q9m35 is a good place to build a mansion. The address is S Grimmer Blvd:Fremont Blvd, Fremont, CA
[Grimmer](Grimmer.jpg)